# 練習

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 108)
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge as R
from sklearn.linear_model import Lasso as L
%matplotlib inline

In [2]:
#データセット読み込み
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
stadium = pd.read_csv("stadium.csv")
sample = pd.read_csv("sample_submit.csv", header=None)
cond = pd.read_csv("condition.csv")
cond_add = pd.read_csv("condition_add.csv")
data_2014 = pd.read_csv("2014_add.csv")
train_add = pd.read_csv("train_add.csv")

In [3]:
cond.head()

,id,home_score,away_score,weather,temperature,humidity,referee,home_team,home_01,home_02,...,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,13994,1,0,雨,3.8,66%,木村 博之,ベガルタ仙台,林 卓人,菅井 直樹,...,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ
1,13995,1,0,屋内,12.4,43%,西村 雄一,名古屋グランパス,楢﨑 正剛,田中 隼磨,...,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀
2,13996,2,3,晴一時雨,11.3,41%,高山 啓義,ガンバ大阪,藤ヶ谷 陽介,加地 亮,...,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人
3,13997,1,0,曇一時雨のち晴,11.4,52%,松尾 一,サンフレッチェ広島,西川 周作,森脇 良太,...,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也
4,13998,0,0,屋内,22.5,32%,廣瀬 格,コンサドーレ札幌,李 昊乗,高木 純平,...,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一


In [4]:
#idをkeyにしてファイルを結合
train = pd.merge(train, cond, left_on="id", right_on="id")
test = pd.merge(test, cond, left_on="id", right_on="id")

In [5]:
train["time"] = train["time"].apply(lambda x: x.split(":")[0])
test["time"] = test["time"].apply(lambda x: x.split(":")[0])
train["time"] = train["time"].astype(float)
test["time"] = test["time"].astype(float)
train["humidity"] = train["humidity"].apply(lambda x: x.split("%")[0])
test["humidity"] = test["humidity"].apply(lambda x: x.split("%")[0])
train["humidity"] = train["humidity"].astype(float)
test["humidity"] = test["humidity"].astype(float)
y = train["y"]
train.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,...,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14.0,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,...,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14.0,名古屋グランパス,清水エスパルス,豊田スタジアム,...,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14.0,ガンバ大阪,ヴィッセル神戸,万博記念競技場,...,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14.0,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,...,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14.0,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,...,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一


In [13]:
df = pd.concat([train, test])
df["week"] = df["gameday"].apply(lambda x: list(x)[-2])
df.head()

df_dummies = pd.get_dummies(df[["week", "stage", "temperature", "humidity", "time", "stadium", "weather"]])
df_dummies.head()

,temperature,humidity,time,week_休,week_土,week_日,week_月,week_木,week_水,week_火,...,weather_曇時々雨,weather_曇時々雨のち晴,weather_雨,weather_雨のち晴,weather_雨のち曇,weather_雨のち曇一時雨,weather_雨のち曇時々晴,weather_雨時々晴,weather_雨時々曇,weather_雪のち雨
0,3.8,66.0,14.0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,12.4,43.0,14.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11.3,41.0,14.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11.4,52.0,14.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22.5,32.0,14.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
trainX = df_dummies.iloc[:1721, :]
testX = df_dummies.iloc[1721:, :]
print(trainX.info())
print(testX.info())
trainX.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721 entries, 0 to 1720
Columns: 107 entries, temperature to weather_雪のち雨
dtypes: float64(3), uint8(104)
memory usage: 228.6 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 0 to 312
Columns: 107 entries, temperature to weather_雪のち雨
dtypes: float64(3), uint8(104)
memory usage: 41.6 KB
None


,temperature,humidity,time,week_休,week_土,week_日,week_月,week_木,week_水,week_火,...,weather_曇時々雨,weather_曇時々雨のち晴,weather_雨,weather_雨のち晴,weather_雨のち曇,weather_雨のち曇一時雨,weather_雨のち曇時々晴,weather_雨時々晴,weather_雨時々曇,weather_雪のち雨
0,3.8,66.0,14.0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,12.4,43.0,14.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11.3,41.0,14.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11.4,52.0,14.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22.5,32.0,14.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
testX.head()

,temperature,humidity,time,week_休,week_土,week_日,week_月,week_木,week_水,week_火,...,weather_曇時々雨,weather_曇時々雨のち晴,weather_雨,weather_雨のち晴,weather_雨のち曇,weather_雨のち曇一時雨,weather_雨のち曇時々晴,weather_雨時々晴,weather_雨時々曇,weather_雪のち雨
0,27.4,70.0,19.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30.8,65.0,18.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,31.7,58.0,19.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,29.3,76.0,19.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30.4,68.0,19.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
trainX.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721 entries, 0 to 1720
Columns: 107 entries, temperature to weather_雪のち雨
dtypes: float64(3), uint8(104)
memory usage: 228.6 KB


In [17]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 0 to 312
Columns: 107 entries, temperature to weather_雪のち雨
dtypes: float64(3), uint8(104)
memory usage: 41.6 KB


In [11]:
model = L()

In [12]:
model.fit(trainX, y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [13]:
model.score(trainX, y)

0.789286037805411

In [24]:
from sklearn.ensemble import RandomForestRegressor as RFR

clf = RFR(max_depth=30, n_estimators=300, random_state=709)
clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))

0.7541047269430264


In [14]:
pred = model.predict(testX)

In [15]:
sample[1] = pred

In [16]:
sample.to_csv("submit11.csv", header=None, index=None)

In [16]:
train.plot.scatter("weather", "y")
train["week"].index

ValueError: scatter requires x column to be numeric

In [20]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

C:\Users\madoc\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
trainX.shape

(1721, 107)

In [21]:
testX.shape

(313, 107)

In [25]:
def create_model():
    
    model = Sequential()
    model.add(Dense(107, input_dim=107, activation="relu"))
    model.add(Dense(214, activation="relu"))
    model.add(Dense(1))

    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

In [21]:
x_train, x_test, y_train, y_test = train_test_split(trainX, y, random_state=709)

In [26]:
estimator = KerasRegressor(build_fn=create_model, epochs=200, batch_size=10, verbose=1)
estimator.fit(x_train, y_train)
y_pred = estimator.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print("keras REG mse = {:.4f}".format(mse ** 0.5))

Epoch 1/200
1290/1290 [==============================] - 1s 391us/step - loss: 151861322.8217
Epoch 2/200
1290/1290 [==============================] - 0s 195us/step - loss: 75212008.8527
Epoch 3/200
1290/1290 [==============================] - 0s 190us/step - loss: 72141545.8992
Epoch 4/200
1290/1290 [==============================] - 0s 184us/step - loss: 70241166.0504
Epoch 5/200
1290/1290 [==============================] - 0s 180us/step - loss: 68692934.1395
Epoch 6/200
1290/1290 [==============================] - 0s 178us/step - loss: 66430700.2868
Epoch 7/200
1290/1290 [==============================] - 0s 216us/step - loss: 64115130.9922
Epoch 8/200
1290/1290 [==============================] - 0s 200us/step - loss: 61768725.9690
Epoch 9/200
1290/1290 [==============================] - 0s 225us/step - loss: 58852195.3721
Epoch 10/200
1290/1290 [==============================] - 0s 213us/step - loss: 55719808.6434
Epoch 11/200
1290/1290 [==============================] - 0s 213us/s

1290/1290 [==============================] - 0s 195us/step - loss: 11674205.6715
Epoch 89/200
1290/1290 [==============================] - 0s 187us/step - loss: 11550724.0610
Epoch 90/200
1290/1290 [==============================] - 0s 192us/step - loss: 11568390.1172
Epoch 91/200
1290/1290 [==============================] - 0s 197us/step - loss: 11784050.2074
Epoch 92/200
1290/1290 [==============================] - 0s 221us/step - loss: 11423168.3159
Epoch 93/200
1290/1290 [==============================] - 0s 221us/step - loss: 11470055.2393
Epoch 94/200
1290/1290 [==============================] - 0s 222us/step - loss: 11422147.7926
Epoch 95/200
1290/1290 [==============================] - 0s 230us/step - loss: 11405263.6986
Epoch 96/200
1290/1290 [==============================] - 0s 222us/step - loss: 11426198.2810
Epoch 97/200
1290/1290 [==============================] - 0s 220us/step - loss: 11323817.2161
Epoch 98/200
1290/1290 [==============================] - 0s 235us/step -

1290/1290 [==============================] - 0s 191us/step - loss: 10108307.7297
Epoch 175/200
1290/1290 [==============================] - 0s 193us/step - loss: 10001577.2546
Epoch 176/200
1290/1290 [==============================] - 0s 230us/step - loss: 10093566.40990s - loss: 9758494
Epoch 177/200
1290/1290 [==============================] - 0s 210us/step - loss: 10233890.1013
Epoch 178/200
1290/1290 [==============================] - 0s 231us/step - loss: 10034158.8556
Epoch 179/200
1290/1290 [==============================] - 0s 221us/step - loss: 9985506.4622
Epoch 180/200
1290/1290 [==============================] - 0s 226us/step - loss: 9945407.8595
Epoch 181/200
1290/1290 [==============================] - 0s 224us/step - loss: 9962247.2578
Epoch 182/200
1290/1290 [==============================] - 0s 198us/step - loss: 10041356.5397
Epoch 183/200
1290/1290 [==============================] - 0s 188us/step - loss: 9917659.3450
Epoch 184/200
1290/1290 [=========================

In [27]:
pred_y_test = estimator.predict(testX)
pred_y_test
estimator.score(x_test, y_test)

431/431 [==============================] - 0s 143us/step


-14896182.13863109

In [37]:
sample[1] = pred_y_test
sample.to_csv("submit_deep.csv", header=None, index=None)

In [24]:
#全データで学習
estimator2 = KerasRegressor(build_fn=create_model, epochs=200, batch_size=10, verbose=1)
estimator2.fit(trainX, y)

pred_y_test = estimator2.predict(testX)
sample[1] = pred_y_test
sample.to_csv("submit_deep2.csv", header=None, index=None)


Epoch 1/200
1721/1721 [==============================] - 1s 318us/step - loss: 136796796.7745
Epoch 2/200
1721/1721 [==============================] - 0s 170us/step - loss: 73929186.6380
Epoch 3/200
1721/1721 [==============================] - 0s 169us/step - loss: 71469494.5729
Epoch 4/200
1721/1721 [==============================] - 0s 179us/step - loss: 69202093.8059
Epoch 5/200
1721/1721 [==============================] - 0s 166us/step - loss: 66884375.3559
Epoch 6/200
1721/1721 [==============================] - 0s 171us/step - loss: 63895742.6078
Epoch 7/200
1721/1721 [==============================] - 0s 229us/step - loss: 60468628.9745
Epoch 8/200
1721/1721 [==============================] - 0s 177us/step - loss: 56589106.2289
Epoch 9/200
1721/1721 [==============================] - 0s 175us/step - loss: 52021982.5633
Epoch 10/200
1721/1721 [==============================] - 0s 176us/step - loss: 47404281.1780
Epoch 11/200
1721/1721 [==============================] - 0s 174us/s

1721/1721 [==============================] - 0s 199us/step - loss: 10632462.3613
Epoch 174/200
1721/1721 [==============================] - 0s 153us/step - loss: 10598539.4658
Epoch 175/200
1721/1721 [==============================] - 0s 197us/step - loss: 10566905.2897
Epoch 176/200
1721/1721 [==============================] - 0s 166us/step - loss: 10604096.0400
Epoch 177/200
1721/1721 [==============================] - 0s 180us/step - loss: 10619160.9783
Epoch 178/200
1721/1721 [==============================] - 0s 196us/step - loss: 10585217.9595
Epoch 179/200
1721/1721 [==============================] - 0s 175us/step - loss: 10534953.5011
Epoch 180/200
1721/1721 [==============================] - 0s 179us/step - loss: 10771251.5529
Epoch 181/200
1721/1721 [==============================] - 0s 203us/step - loss: 10586355.8469
Epoch 182/200
1721/1721 [==============================] - 0s 173us/step - loss: 10542418.1736
Epoch 183/200
1721/1721 [==============================] - 0s 16

In [23]:
def create_model2():
    model = Sequential()
    
    model.add(Dense(107, input_dim=107))
    model.add(Activation("relu"))
    
    model.add(Dense(428))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    
    model.add(Dense(107))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    
    model.add(Dense(1))
    
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

In [31]:
estimator3 = KerasRegressor(build_fn=create_model2, epochs=200, batch_size=10, verbose=1)
estimator3.fit(x_train, y_train)
y_pred = estimator.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print("keras REG mse = {:.4f}".format(mse ** 0.5))

Epoch 1/200
1290/1290 [==============================] - 1s 1ms/step - loss: 115240920.1783
Epoch 2/200
1290/1290 [==============================] - 1s 400us/step - loss: 75751889.7054
Epoch 3/200
1290/1290 [==============================] - 0s 337us/step - loss: 69862576.9302
Epoch 4/200
1290/1290 [==============================] - 0s 346us/step - loss: 61528334.3333
Epoch 5/200
1290/1290 [==============================] - 0s 354us/step - loss: 54375804.9845
Epoch 6/200
1290/1290 [==============================] - 0s 374us/step - loss: 46641550.5349
Epoch 7/200
1290/1290 [==============================] - 0s 383us/step - loss: 41405232.5465
Epoch 8/200
1290/1290 [==============================] - 0s 323us/step - loss: 39153577.6667
Epoch 9/200
1290/1290 [==============================] - 0s 361us/step - loss: 34618110.2171
Epoch 10/200
1290/1290 [==============================] - 0s 344us/step - loss: 33191587.7694
Epoch 11/200
1290/1290 [==============================] - 0s 385us/ste

1290/1290 [==============================] - 0s 270us/step - loss: 12052585.8333
Epoch 174/200
1290/1290 [==============================] - 0s 319us/step - loss: 11897330.3333
Epoch 175/200
1290/1290 [==============================] - 0s 300us/step - loss: 12949994.9695
Epoch 176/200
1290/1290 [==============================] - 0s 288us/step - loss: 12981316.9457
Epoch 177/200
1290/1290 [==============================] - 0s 309us/step - loss: 11468371.1134
Epoch 178/200
1290/1290 [==============================] - 0s 293us/step - loss: 11632491.4070
Epoch 179/200
1290/1290 [==============================] - 0s 273us/step - loss: 12132118.9603
Epoch 180/200
1290/1290 [==============================] - 0s 323us/step - loss: 11104430.1754
Epoch 181/200
1290/1290 [==============================] - 0s 314us/step - loss: 11234133.9370
Epoch 182/200
1290/1290 [==============================] - 0s 300us/step - loss: 10086166.7297
Epoch 183/200
1290/1290 [==============================] - 0s 30

In [24]:
estimator4 = KerasRegressor(build_fn=create_model2, epochs=200, batch_size=10, verbose=1)
estimator4.fit(trainX, y)

mse = mean_squared_error(y_test, y_pred)
print("keras REG mse = {:.4f}".format(mse ** 0.5))

pred_y_test = estimator4.predict(testX)
sample[1] = pred_y_test
sample.to_csv("submit_deep4.csv", header=None, index=None)


Epoch 1/200
1721/1721 [==============================] - 1s 656us/step - loss: 106076523.7095
Epoch 2/200
1721/1721 [==============================] - 1s 408us/step - loss: 72913642.2388
Epoch 3/200
1721/1721 [==============================] - 1s 466us/step - loss: 65310017.2202
Epoch 4/200
1721/1721 [==============================] - 1s 385us/step - loss: 57555987.3306
Epoch 5/200
1721/1721 [==============================] - 1s 385us/step - loss: 46809183.6426
Epoch 6/200
1721/1721 [==============================] - 1s 477us/step - loss: 40342691.2289
Epoch 7/200
1721/1721 [==============================] - 1s 381us/step - loss: 36781617.6065
Epoch 8/200
1721/1721 [==============================] - 1s 385us/step - loss: 33693664.1782
Epoch 9/200
1721/1721 [==============================] - 1s 446us/step - loss: 31503948.2284
Epoch 10/200
1721/1721 [==============================] - 1s 413us/step - loss: 30267411.2377
Epoch 11/200
1721/1721 [==============================] - 1s 421us/s

1721/1721 [==============================] - 1s 376us/step - loss: 12494258.4396
Epoch 174/200
1721/1721 [==============================] - 1s 429us/step - loss: 14364822.2856
Epoch 175/200
1721/1721 [==============================] - 1s 380us/step - loss: 13960879.4972
Epoch 176/200
1721/1721 [==============================] - 1s 400us/step - loss: 13389235.1720
Epoch 177/200
1721/1721 [==============================] - 1s 389us/step - loss: 13213424.9825
Epoch 178/200
1721/1721 [==============================] - 1s 397us/step - loss: 14492595.4791
Epoch 179/200
1721/1721 [==============================] - 1s 390us/step - loss: 13420844.1569
Epoch 180/200
1721/1721 [==============================] - 1s 407us/step - loss: 13023110.5751
Epoch 181/200
1721/1721 [==============================] - 1s 447us/step - loss: 12176277.8171
Epoch 182/200
1721/1721 [==============================] - 1s 394us/step - loss: 12125942.3614
Epoch 183/200
1721/1721 [==============================] - 1s 41

In [34]:
def create_model3():
    model = Sequential()
    
    model.add(Dense(107, input_dim=107))
    model.add(Activation("relu"))
    
    model.add(Dense(214))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    
    model.add(Dense(1))
    
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

In [35]:
estimator5 = KerasRegressor(build_fn=create_model3, epochs=100, batch_size=10, verbose=1)
estimator5.fit(trainX, y)

pred_y_test = estimator5.predict(testX)
sample[1] = pred_y_test
sample.to_csv("submit_deep5.csv", header=None, index=None)


Epoch 1/100
1721/1721 [==============================] - 1s 476us/step - loss: 142296689.8132
Epoch 2/100
1721/1721 [==============================] - 0s 242us/step - loss: 75444251.3643
Epoch 3/100
1721/1721 [==============================] - 0s 237us/step - loss: 73619589.7048
Epoch 4/100
1721/1721 [==============================] - 0s 235us/step - loss: 70776009.4805
Epoch 5/100
1721/1721 [==============================] - 0s 257us/step - loss: 69242530.5619
Epoch 6/100
1721/1721 [==============================] - 0s 244us/step - loss: 67191508.4776
Epoch 7/100
1721/1721 [==============================] - 0s 233us/step - loss: 62403182.3719
Epoch 8/100
1721/1721 [==============================] - 0s 285us/step - loss: 60982465.6436
Epoch 9/100
1721/1721 [==============================] - 1s 295us/step - loss: 56151983.8189
Epoch 10/100
1721/1721 [==============================] - 0s 249us/step - loss: 52498958.4782
Epoch 11/100
1721/1721 [==============================] - 0s 243us/s

In [47]:
trainX.to_csv("trainX.csv", index=None)
testX.to_csv("testX.csv", index=None)
y.to_csv("y.csv",index=None)

In [23]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=4, random_state=709)

regr.fit(x_train, y_train)
print(regr.score(x_train, y_train))
print(regr.score(x_test, y_test))

0.7269910061243314
0.6648798031555441


In [ ]:
###GridSearch